In [8]:
import pandas as pd

In [9]:
import csv

# file_path = 'data/rum_generated.csv'
df = pd.read_csv("../data/rum_generated.csv")
df.head(10)



,user_id,country,user_action,timestamp,readable_timestamp
0,2540,Egypt,join_plan,1754268121000,2025-08-04 01:42:01 AM
1,2540,Egypt,sent_message,1754313104000,2025-08-04 02:11:44 PM
2,2540,Egypt,visit_home,1754328169000,2025-08-04 06:22:49 PM
3,2540,Egypt,visit_plans,1754311339000,2025-08-04 01:42:19 PM
4,8963,Nigeria,visit_plans,1753574553000,2025-07-27 01:02:33 AM
5,8273,Egypt,join_plan,1753585569000,2025-07-27 04:06:09 AM
6,8273,Egypt,visit_home,1753576615000,2025-07-27 01:36:55 AM
7,8273,Egypt,visit_home,1753632667000,2025-07-27 05:11:07 PM
8,1002,South Africa,visit_plans,1753204347000,2025-07-22 06:12:27 PM
9,1002,South Africa,visit_plans,1753140668000,2025-07-22 12:31:08 AM


In [10]:
#Inspect df
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             4809 non-null   int64 
 1   country             4809 non-null   object
 2   user_action         4809 non-null   object
 3   timestamp           4809 non-null   int64 
 4   readable_timestamp  4809 non-null   object
dtypes: int64(2), object(3)
memory usage: 188.0+ KB


(4809, 5)

In [11]:
#Checking for missing data or Duplicates

#missing data
print(df.isnull().sum())

#duplicates
print(f'Duplicates: {df.duplicated().sum()}')

user_id               0
country               0
user_action           0
timestamp             0
readable_timestamp    0
dtype: int64
Duplicates: 0


In [12]:
print(df['country'].value_counts())
print(df['user_action'].value_counts())


country
UK              895
USA             846
South Africa    817
Nigeria         796
Ghana           728
Egypt           727
Name: count, dtype: int64
user_action
visit_home          2995
visit_plans          927
visit_trainers       291
join_plan            209
subscribed_basic     126
sent_message         116
subscribed_pro        81
subscribed_elite      64
Name: count, dtype: int64


In [13]:
# 1) Make sure column is string and do quick cleaning
df['readable_timestamp'] = df['readable_timestamp'].astype(str).str.strip()
# remove trailing dots or accidental characters like '.' at end
df['readable_timestamp'] = df['readable_timestamp'].str.replace(
    r'\.+$', '', regex=True)
# remove duplicate spaces
df['readable_timestamp'] = df['readable_timestamp'].str.replace(
    r'\s+', ' ', regex=True)

# 2) Attempt parsing with a list of likely formats
formats_to_try = [
    "%y-%m-%d %I:%M:%S %p",   # e.g. 25-05-25 05:17:31 AM
    "%Y-%m-%d %I:%M:%S %p",   # e.g. 2025-05-25 05:17:31 AM
    "%m/%d/%Y, %I:%M %p",     # e.g. 10/29/2025, 09:45 AM
    "%m/%d/%y, %I:%M %p",     # e.g. 10/29/25, 09:45 AM
    "%y-%m-%d %H:%M:%S",      # fallback 24-hour without AM/PM
    "%Y-%m-%d %H:%M:%S"
]

# initialize a column of NaT datetimes
df['readable_dt'] = pd.NaT

for fmt in formats_to_try:
    mask = df['readable_dt'].isna()
    try:
        parsed = pd.to_datetime(
            df.loc[mask, 'readable_timestamp'], format=fmt, errors='coerce')
        df.loc[mask, 'readable_dt'] = parsed
    except Exception:
        # if a format causes unexpected error, skip it
        pass

# 3) Final fallback: let pandas infer format for remaining NaT rows
mask = df['readable_dt'].isna()
if mask.any():
    df.loc[mask, 'readable_dt'] = pd.to_datetime(
        df.loc[mask, 'readable_timestamp'], errors='coerce', infer_datetime_format=True)

# 4) Inspect how many failed to parse
failed = df['readable_dt'].isna().sum()
print(f"Parsing failures (still NaT): {failed} out of {len(df)}")

# Optionally show some problematic examples (up to 10)
if failed:
    print(df.loc[df['readable_dt'].isna(),
          'readable_timestamp'].head(10).to_list())

# 5) Create separate columns once parsed
# If you want date as ISO (YYYY-MM-DD) for Power BI, use dt.date or dt.strftime
df['date_only'] = df['readable_dt'].dt.date
df['time_only'] = df['readable_dt'].dt.strftime('%I:%M:%S %p')  # keeps AM/PM
df['hour'] = df['readable_dt'].dt.hour
df['day'] = df['readable_dt'].dt.day_name()
df['month'] = df['readable_dt'].dt.month_name()

Parsing failures (still NaT): 0 out of 4809


In [14]:
# Export cleaned data
cleaned_path = '../data/gym_rum_data_cleaned.csv'
df.to_csv(cleaned_path, index=False)
print('Cleaned data saved:', cleaned_path)

Cleaned data saved: ../data/gym_rum_data_cleaned.csv
